In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('max_rows', 7)
train_df_original = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv',
                                index_col=0)
train_df_original

In [ ]:
test_df_original = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv',
                              index_col=0)
test_df_original

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(5, 3, figsize=(15, 25))

columns = iter(list((train_df_original.columns)))


for i in range(5):
    for j in range(3):
        sns.histplot(train_df_original[next(columns)], ax=ax[i, j])

In [ ]:
train_df = train_df_original.copy()
test_df = test_df_original.copy()

y = train_df.pop('target')
X = train_df.drop(['cat0', 'cat2', 'cat4', 'cat6', 'cat7'], axis=1)
test_df.drop(['cat0', 'cat2', 'cat4', 'cat6', 'cat7'], axis=1, inplace=True)
test_data = pd.get_dummies(test_df)
X = pd.get_dummies(X)
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

def mae_error(max_leaf_nodes, X_train, X_valid, y_train, y_valid):
    dtr_model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes)
    dtr_model.fit(X_train, y_train)
    y_pred = dtr_model.predict(X_valid)
    mae = mean_absolute_error(y_pred, y_valid)
    return mae
    
    
mln_list = [5, 50, 500, 5000]
mae_list = []
for i in mln_list:
    my_mae = mae_error(i, X_train, X_valid, y_train, y_valid)
    mae_list.append(my_mae)
    print(i, my_mae)
    
sns.lineplot(mln_list, mae_list)  

In [ ]:
final_model = DecisionTreeRegressor(max_leaf_nodes=50)
final_model.fit(X, y)

final_pred = final_model.predict(test_data)

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv',
                        index_col=0)
submission

In [ ]:
submission['target'] = final_pred
submission.to_csv('submission.csv')